# Setting up Hyperparameters

In [ ]:
load_data_on_GPU = False
nb_augment = 3

epochs = 20
batch_size = 4
learning_rate = 1e-4

### Selecting the correct device for training (CPU or GPU)

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Importing the Data

In [ ]:
from dataset import POCDataReader, data_augment_

data_reader = POCDataReader(root_dir="../data/POC", load_on_gpu=False, limit=None, verbose=True)
train_data, val_data, test_data = data_reader.split()

train_data = data_augment_(train_data, n=nb_augment, load_on_gpu=False, verbose=True, seed=1234)

In [ ]:
from torch.nn import Sequential
from torchvision.transforms.functional import invert
from torch.nn.functional import normalize

from pipelines import InputPipeline, SumFilters
from pipelines.filters import *


sumFilter = SumFilters(FrangiFilter(), SatoFilter())

inpip = InputPipeline(
    transformer=None,
    layer_transformer=[
        LaplacianFilter(),
#         FrangiFilter(),
#         SatoFilter(),
#         sumFilter,
   ]
)

# inpip = inpip.to(device)

### Creating Training Dataset

In [ ]:
from torch.nn.functional import normalize
from torchvision.transforms import CenterCrop, Resize, GaussianBlur, RandomCrop

from dataset import POCDataset

train_dataset = POCDataset(
    train_data,
    transform=inpip,
    target_transform=None,
    negative_mining=False,
    load_on_gpu=load_data_on_GPU,
    verbose=True
)

train_dataset.precompute_transform(verbose=True)

In [ ]:
from torch.utils.data import DataLoader

if load_data_on_GPU:
    training_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=train_dataset.sampler,
        shuffle= True if train_dataset.sampler is None else None,
    )
else:
    training_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=train_dataset.sampler,
        shuffle= True if train_dataset.sampler is None else None,
        num_workers=8,
        pin_memory=True,
        pin_memory_device=device,
    )

### Creating Validation Dataset

In [ ]:
from dataset import POCDataset

val_dataset = POCDataset(val_data, transform=inpip, target_transform=None, negative_mining=False)

In [ ]:
from torch.utils.data import DataLoader

if load_data_on_GPU:
    validation_dataloader = DataLoader(
        val_dataset,
        batch_size= 2 * batch_size,
        shuffle=True,
    )
else:
    validation_dataloader = DataLoader(
        val_dataset,
        batch_size= 2 * batch_size,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        pin_memory_device=device,
    )

### Creating Evaluation Dataset

In [ ]:
from dataset import POCDataset

test_dataset = POCDataset(test_data, transform=inpip, target_transform=None, negative_mining=False)

In [ ]:
from torch.utils.data import DataLoader

if load_data_on_GPU:
    evaluation_dataloader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=True,
    )
else:
    evaluation_dataloader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=8,
        pin_memory=True,
        pin_memory_device=device,
    )

### Testing the dataloader

In [ ]:
from my_utils import show_img

features, masks, files, indexes = next(iter(training_dataloader))
print(features.size(), masks.size())

show_img(features)
show_img(masks)
print(files, indexes)

### Testing Dataset for proportion between classes

In [ ]:
_,train_labels,_,_ = next(iter(training_dataloader))

nb_pixel = torch.unique(train_labels, return_counts=True)[1]
print("Proportion of class 1 in this batch: {}%".format(nb_pixel[1] * 100 / (nb_pixel[0] + nb_pixel[1])))

Result is usually around 1.1% which may be too low for cross_entropy_loss.

# Building the differents modules

## Creating the models

#### Using Unet with bilinear upsampling and cropping to generate 2 classes (background and crack).

In [ ]:
from models import UNet, DeepCrack, SubUNet

# model = UNet(n_channels=inpip.nb_channel, n_classes=2, bilinear=True, crop=False).to(device)
# model = DeepCrack(n_channels=inpip.nb_channel, n_classes=2).to(device)
model = SubUNet(n_channels=inpip.nb_channel, n_classes=2).to(device)

In [ ]:
print("Total number of parameters: {0:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print(f"Model structure: {model}")

## Creating the loss functions (Choose which one you need)

#### Cross Entropy Loss

In [ ]:
import torch
from torch.nn import CrossEntropyLoss

weight = torch.tensor([.9, .1])  # class weight : 98% for background and 2% for the cracks
loss_fn = CrossEntropyLoss(weight=weight).to(device)

#### Focal Loss

In [ ]:
import torch
from loss import FocalLoss

weight = torch.tensor([.9, .1])  # class weight : 90% for background and 10% for the cracks
loss_fn = FocalLoss(weight=weight, gamma=2).to(device)

#### Jaccard Loss

In [ ]:
from loss import JaccardLoss

loss_fn = JaccardLoss().to(device)

#### Tversky Loss

In [ ]:
import torch
from loss import TverskyLoss

loss_fn = TverskyLoss(alpha=0.3, beta=0.7).to(device)

#### Focal Tversky Loss

In [ ]:
import torch
from loss import FocalTverskyLoss

loss_fn = FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=2).to(device)

#### MeanLoss (Focal + Tversky)

In [ ]:
import torch
from loss import MeanLoss, TverskyLoss, FocalLoss

weight = torch.tensor([.3, .7])

loss_fn = MeanLoss(
    FocalLoss(weight=weight, gamma=2),
    TverskyLoss(alpha=0.3, beta=0.7),
    ratio=0.75).to(device)

##### MultiscaleLoss will allow the loss function to work on tuple of outputs instead of a single segmentation map (Used only for DeepCrack Network)

In [ ]:
# from loss import MultiscaleLoss

# loss_fn = MultiscaleLoss(loss_fn)

## Creating the Optimizer

In [ ]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.99))

## Setting up the Learning Rate Scheduler

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

lr_scheduler = CosineAnnealingLR(optimizer, T_max=epochs//3)

## Creating the Metrics
Can be automated but I set it to manualy select because I needed a quick and simple use once

In [ ]:
from metrics import Metrics, EvaluationMetrics


config = {
    "Network": model.__class__,
    "Optimizer": optimizer.__class__,

    "Learning Rate": learning_rate,
    "Batch Size": batch_size,

    "Loss Combiner": loss_fn.__class__,
    "Loss Combiner_ratio": loss_fn.ratio,
    "Loss Volume": loss_fn.loss1,
    "Loss Pixel": loss_fn.loss2,

    "Pipe Filter": inpip.filter,
    "Pipe Layer": inpip.additional_channel,
}

train_metrics = Metrics(
    buffer_size=len(training_dataloader),
    mode="Training",
    hyperparam=config,
    device=device)

val_metrics = Metrics(
    buffer_size=len(validation_dataloader),
    mode="Validation",
    hyperparam=config,
    device=device)

test_metrics = EvaluationMetrics(
    buffer_size=len(evaluation_dataloader),
    hyperparam=config,
    epochs=epochs,
    device=device)

# Training, testing and validating the Model

In [ ]:
from train_tqdm import training_loop, validation_loop, evaluation_loop

for epoch in range(1, epochs+1):
    training_loop(epoch, training_dataloader, model, loss_fn, optimizer, lr_scheduler, train_metrics, device)
    validation_loop(epoch, validation_dataloader, model, loss_fn, val_metrics, device)

evaluation_loop(evaluation_dataloader, model, test_metrics, device)

train_metrics.close_tensorboard()
val_metrics.close_tensorboard()
test_metrics.close_tensorboard()